# 플젠 - 창업 / 사이드 프로젝트 아이디어 생성 AI v0.1

플젠은 새로운 프로젝트 아이디어가 떠오르지 않을 때 사용자의 특성과 시장성 등을 고려해 적합한 아이디어 제안해 주는 AI 입니다.

v0.1 에서는 사용자가 입력한 키워드에 대해 아이디어를 생성하는 기능까지 구현했습니다.

## 1. 데이터 수집



### 1-1. N 플랫폼
스타트업 소개, 투자유치, 제품 정보 제공 플랫폼에서 데이터

#### 전체 1만8천개 스타트업 URL 수집


In [ ]:
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install webdriver-manager

In [ ]:
from selenium import webdriver
import chromedriver_autoinstaller
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.webdriver import ActionChains
from tqdm import tqdm_notebook as tq

In [ ]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())
caps = DesiredCapabilities.CHROME
caps["pageLoadStrategy"] = "none"

In [ ]:
# 넥스트 유니콘 로그인 및 파인더 페이지 진입
try:
    crawler = webdriver.Chrome(options=chrome_options)
except:
    chromedriver_autoinstaller.install(True)
    crawler = webdriver.Chrome(options=chrome_options)
crawler.get('https://www.nextunicorn.kr/login')
time.sleep(1)

# 로그인
inputid = crawler.find_element(By.CSS_SELECTOR, "#__next > div.css-hykhfv.e1qaxoiy0 > div > div > div.css-p0yqi0.ezpvhaw0 > div > div > div > form > div:nth-child(1) > div > div > div > input")
inputid.send_keys("___아이디 입력란___")

inputpw = crawler.find_element(By.CSS_SELECTOR, "#__next > div.css-hykhfv.e1qaxoiy0 > div > div > div.css-p0yqi0.ezpvhaw0 > div > div > div > form > div:nth-child(3) > div > div > div > input")
inputpw.send_keys("___PW 입력란___")

time.sleep(1)
loginBtn = crawler.find_element(By.CSS_SELECTOR, "#__next > div.css-hykhfv.e1qaxoiy0 > div > div > div.css-p0yqi0.ezpvhaw0 > div > div > div > form > button")
loginBtn.click()

# 팝업창 닫기
time.sleep(3)
crawler.find_element(By.CLASS_NAME, 'css-v6ukax').click()

time.sleep(1)
finderBtn = crawler.find_element(By.CSS_SELECTOR, '#menu-finder').click()

기업 페이지 url 수집

In [ ]:
total_index = list(range(1, 1201))
url_list = []

try:
    for i in tq(total_index):

        # 더보기 버튼 클릭
        if i != 0 and i % 12 == 0:
            view_mort_btn = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-w0y1nt-2.hyyQzY > div > div > div.sc-wmx587-0.kZYyTF > div > div:nth-child(3) > div.sc-wmx587-2.gipIFo > button')
            view_mort_btn.click()
            time.sleep(2)
        else:
            pass

        # 기업명, URL 수집
        try:
            link = crawler.find_element(By.CSS_SELECTOR, f'#modal-wrapper > div > div.sc-w0y1nt-2.hyyQzY > div > div > div.sc-wmx587-0.kZYyTF > div > div:nth-child(3) > div.sc-wmx587-2.gipIFo > div.sc-1xk12lb-0.dbdPec > a:nth-child({i})').get_attribute('href')
            title = crawler.find_element(By.XPATH, f'//*[@id="modal-wrapper"]/div/div[2]/div/div/div[3]/div/div[3]/div[2]/div[3]/a[{i}]/div/div[2]/div[1]/div[1]/span').text
        except:
            link = 0
            title = 0

        url_list.append([title, link])

except:
    pass

# URL 저장
data_set = pd.DataFrame(url_list, columns = ['회사명', 'url'])
data_set.to_csv("___파일 경로___", index=False)

기업 상세페이지 내 데이터 수집
- 회사명 title
- 한줄소개 desc
- 회사정보 company_info
- 설립일 company_date
- 홈페이지 company_homepage
- 카테고리 company_category
- 기술 company_tech
- 누적투자금 company_funding
- 최근투자일 company_funding_date
- 투자라운드 company_stage
- 제품소개 company_pruduct
- url company_current_url

In [ ]:
url_list = pd.read_csv("___파일 경로___")

crawler.implicitly_wait(0.5)
result = []
len_url = range(len(url_list))

try:
    for i in tq(len_url):

        # url 접속
        crawler.get(url_list['url'][i])
        time.sleep(1)

        # 기본정보 수집
        title = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1au6rf5-0.dfyEHV > div.sc-1tgp7us-0.kkEMVy > div > div > div.sc-1jo1n25-0.eKTDmM > h1').text
        try:
            desc = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1au6rf5-0.dfyEHV > div.sc-1tgp7us-0.kkEMVy > div > div > div.sc-1jo1n25-0.eKTDmM > div > p').text
        except:
            desc = 0
        try:
            company_info = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1d5h8cg-0.keYmOz > div > section.sc-tzw65z-0 > div').text
        except:
            company_info = 0
        try:
            company_date = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1f7t804-0.bMzzFA > div.sc-sku89p-0.imGnwh > div.sc-sku89p-2.gapIhE > div:nth-child(2) > div:nth-child(2) > div.sc-15mb17d-2.kPZzPy').text
        except:
            company_date = 0
        try:
            company_homepage = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1f7t804-0.bMzzFA > div.sc-sku89p-0.imGnwh > div.sc-sku89p-2.gapIhE > div:nth-child(2) > div:nth-child(3) > a').get_attribute("href")
        except:
            company_homepage = 0
        try:
            company_category = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1f7t804-0.bMzzFA > div.sc-sku89p-0.imGnwh > div.sc-sku89p-2.gapIhE > div.sc-1sr3kif-0.efXkfK').text
        except:
            company_category = 0
        try:
            company_tech = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1f7t804-0.bMzzFA > div.sc-sku89p-0.imGnwh > div.sc-sku89p-2.gapIhE > div.sc-1lxj15l-0.hgaLMa').text
        except:
            company_tech = 0

        # 기업 페이지 > 투자 페이지 진입
        investBtn = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-zm58l9-0.kIQHCV > div > button:nth-child(2)')
        investBtn.click()

        # 기업 투자 정보 수집
        try:
            company_funding = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1d5h8cg-0.keYmOz > section > div.sc-fxsdks-0.jcsboE > div:nth-child(1) > div.sc-fxsdks-2.iiScRC').text
        except:
            company_funding = 0
        try:
            company_funding_date = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1d5h8cg-0.keYmOz > section > div.sc-1ew9kdj-0.ejNJNv > div > div.sc-1g6qmza-3.iPSlxv > div.sc-1g6qmza-6.fjsLvm').text
        except:
            company_funding_date = 0
        try:
            company_stage = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1d5h8cg-0.keYmOz > section > div.sc-fxsdks-0.jcsboE > div:nth-child(2) > div.sc-fxsdks-2.iiScRC').text
        except:
            company_stage = 0

        # 기업 페이지 > 제품 페이지 진입
        productBtn = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-zm58l9-0.kIQHCV > div > button:nth-child(3)')
        productBtn.click()

        # 기업 제품 정보 수집
        try:
            company_pruduct = crawler.find_element(By.CSS_SELECTOR, '#modal-wrapper > div > div.sc-1tgp7us-0.sc-c899et-1.kkEMVy.cSpElH > div.sc-1d5h8cg-0.keYmOz > section > section > div > div > button').text
        except:
            company_pruduct = 0

        company_current_url = url_list['url'][i]

        # 데이터 저장
        result.append([title, desc, company_info, company_date, company_homepage, company_category, company_tech, company_funding, company_funding_date, company_stage, company_pruduct, company_current_url])
except:
    pass

# csv 저장
data_set = pd.DataFrame(result, columns = ['회사명', '한줄소개', '회사정보', '설립일', '홈페이지', '카테고리', '기술', '누적투자금', '최근투자일', '투자라운드', '제품소개', 'url'])
data_set.to_csv("/Users/yug/Downloads/nextunicorn_data_full.csv", index=False)

1-2. L 플랫폼: 신규 어플 중심 DB (200여개 데이터 수집)

In [ ]:
!pip install beautifulsoup4
!pip install selenium

import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver

url = 'https://letspl.me/product'

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)
driver.get(url)

# scroll down to the end of the mainpage
max_try = 30
for _ in range(max_try):
    body = driver.find_element(By.CSS_SELECTOR, "body")
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

# screen shot to check if it's the bottom of the site
driver.save_screenshot('webpage.png')

# click to access each product's data (product name, category, information)
for i:
    driver.find_element(By.XPATH, '//*[@id="__next"]/section/div[2]/div[6]/div[2]/f"div{i}/div[3]/div[1]/div[2]/button').click()
    product = driver.find_elements(By.CLASS_NAME, 'expoTitleWrap')
    category = driver.find_elements(By.CLASS_NAME, 'expoBooth')
    info = driver.find_elements(By.CLASS_NAME, 'expoServiceInfoBooth')
    driver.back()
## access failed due to the UI of the site -> data was extracted manually

1-3. D 플랫폼: 신규 어플 중심 DB (200여개 데이터 수집)

In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
options.add_argument('user-agent=' + user_agent)

driver = webdriver.Chrome(options=options)

url = 'https://disquiet.io/?type=product'
driver = Chrome()
driver.get(url)

texts = []

data = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div[3]').text
driver.back()
texts.append(data)
print(data)

texts.apply(lambda x : x.split('프로덕트에 대한 자세한 설명' and '사용해 보러 가기'))

## 2. 데이터 전처리

In [ ]:
import pandas as pd
from konlpy.tag import Okt

#파일 불러오기
product_df = pd.read_csv('merged_file.csv', encoding='utf-8')
product_df

#결측치 확인 및 제거
product_df.isnull().sum()

product_df.dropna(inplace=True)

#Info 토큰화
from konlpy.tag import Okt
import re

# Okt 객체 생성
okt = Okt()

# Okt를 사용한 토큰화 함수
def okt_tokenize(text):
    text = text.lower()
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    tokens = okt.morphs(text, stem=True)
    return tokens

# 텍스트에서 줄바꿈 문자를 제거하는 함수
def remove_newlines(text):
    # 줄바꿈 문자를 공백으로 대체
    cleaned_text = re.sub(r'\n+', ' ', text)
    return cleaned_text

# 텍스트 처리 전에 줄바꿈 문자 제거 적용
product_df['info'] = product_df['info'].apply(remove_newlines)

# Okt 토큰화 적용
product_df['tokenized_info'] = product_df['info'].apply(okt_tokenize)

product_df['tokenized_info'].head()

# 불용어 제거
with open('한국어불용어.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

def remove_stopwords_and_single_chars(tokens):
    return [token for token in tokens if token not in stopwords and len(token) > 1]

product_df['cleaned_info'] = product_df['tokenized_info'].apply(remove_stopwords_and_single_chars)
product_df['cleaned_info'].head(20)

#카테고리 불용어 제거
stopwords = ["기타"]

# Function to clean and split the category column
def clean_and_split(category):
    # Use regular expression to remove special characters and split by spaces
    parts = re.findall('[가-힣a-zA-Z0-9&]+', category)

    # 불용어 제거
    parts = [word for word in parts if word not in stopwords]

    return ', '.join(set(parts))

# Apply the function to each row in the 'category' column and create 'cleaned_cat' column
product_df['cleaned_cat'] = product_df['category'].apply(clean_and_split)
product_df['cleaned_cat'].tail(20)


## ======= detele this part ======= ##
## remain all tokens/words excluding some stopwords such as "기타", "제조"
#카테고리 뒤에서 3개만 저장코드
def extract_last_two_categories(category):
    parts = category.split(', ')  # 쉼표와 공백으로 분할
    if len(parts) >= 2:
        return ', '.join(parts[-3:])  # 마지막 두 부분만 추출하여 결합
    else:
        return category  # 두 개 미만의 요소가 있는 경우 원래의 카테고리 반환
product_df['cleaned_c'] = product_df['cleaned_cat'].apply(extract_last_two_categories)
product_df['cleaned_c'].head(20)
## ======= detele this part ======= ##


#파일 저장
product_df.to_csv('merged_file.csv', index=False)

#필요없는 컬럼 삭제
# 원본 데이터 파일 불러오기
file_path = 'merged_file.csv'
product_df = pd.read_csv(file_path)

# 필요한 데이터프레임 조작 수행
product_df.drop(['cleaned_c'], axis=1, inplace=True)

# 수정된 데이터프레임을 원본 파일에 다시 저장 (덮어쓰기)
product_df.to_csv(file_path, index=False)

#받은 데이터 정리 및 합치기
df = pd.read_csv('nextunicorn_data_240109_1.csv', encoding='utf-8')

# 필요한 컬럼들을 하나의 컬럼으로 합치기
df['info'] = df['한줄소개'] + " " + df['회사정보'] + " " + df['제품소개']

# 카테고리와 기술을 합쳐 하나의 컬럼으로 만들기
df['category'] = df['카테고리'] + " " + df['기술']

df['product'] = df['회사명']

# 결과 확인
df[['info', 'category', 'product']].head()

selected_columns = ['info', 'category', 'product']
df[selected_columns].to_csv('file3.csv', encoding='utf-8', index=False)

# 파일 합치기
df1 = pd.read_csv('file.csv', encoding='utf-8')

df2 = pd.read_csv('file2.csv', encoding='utf-8')

df3 = pd.read_csv('file3.csv', encoding='utf-8')

merged_df = pd.concat([df1, df2, df3])

# 합쳐진 파일 저장
merged_df.to_csv('merged_file.csv', encoding='utf-8', index=False)

## 3. 아이디어 생성 모델

In [ ]:
!pip install konlpy
!pip install summa
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=cb68338d81031daf1db6ea6dec12659ad2973f8d79246cf9dbc8c84a0dee99ab
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
import re
import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from summa import keywords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Bidirectional, Dropout, GRU
from gensim.models import Word2Vec
from gensim.models import FastText as FT
from gensim.models.keyedvectors import KeyedVectors
okt = Okt()

In [ ]:
tokenizer = Tokenizer()
data = pd.read_csv('merged_file.csv', encoding='utf-8')
text = data['cleaned_info'].apply(lambda a : re.findall('[가-힣a-z]+', a))

In [ ]:
data

In [ ]:
text.apply(lambda a : a.append('<eos>'))

In [ ]:
tokenizer.fit_on_texts(text)

In [ ]:
vocab_n = len(tokenizer.word_index) + 1
vocab_n

In [ ]:
embeded = tokenizer.texts_to_sequences(text)

In [ ]:
pad_len = 10
def Padding (text) :
    lists = []
    for ind in range(1, len(text)) :
        if ind < pad_len + 1 :
            words = text[:ind + 1]
        else :
            words = text[ind - pad_len : ind + 1]
        lists.append(words)
    return pad_sequences(lists, pad_len + 1, padding = 'pre')

In [ ]:
t_data = list(map(Padding, embeded))

In [ ]:
data = []
for i in t_data :
    for j in i :
        data.append(j)

In [ ]:
X = []
y = []
for i in data :
    y.append(np.array(list(i).pop(-1)))
    X.append(i[:-1])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
input = tf.keras.Input(shape=(pad_len, ))
x = tf.keras.layers.Embedding(vocab_n, 256)(input)
x = tf.keras.layers.LSTM(512, return_sequences = False)(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
output = tf.keras.layers.Dense(vocab_n, activation = 'softmax')(x)

model = tf.keras.Model(inputs = input, outputs = output)

In [ ]:
lr = 0.05
# lr = tf.keras.optimizers.schedules.ExponentialDecay(
#     0.01, decay_steps=2, decay_rate=0.5, staircase=True)

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
              metrics = ['acc'])

In [ ]:
model.fit(X, y, epochs = 50, batch_size = 4096)

In [ ]:
def sentence_generation(model, tokenizer, current_word): # 모델, 토크나이저, 키워드(단어 여러개 가능)
    current_word = current_word.split(' ')

    # n번 반복
    for _ in range(30):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])
        encoded = pad_sequences(encoded, maxlen=pad_len, padding='pre')

        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        pred = model.predict(encoded)

        top_k = np.argsort(pred[0])[:-4:-1]
        ind = np.random.choice([0,0,0,0,1,1,2]) # random으로 상위 ind위의 단어를 result로 할당한다
        result = tokenizer.index_word.get(top_k[ind])

        if result == 'eos' :
            return current_word
        words = result.split(' ')

        if len(words) != 1 and result == words[-1] :
            continue
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word.append(result)

    return current_word

In [ ]:
from summa import keywords
import gensim
from konlpy.tag import Hannanum, Okt, Kkma
from gensim.models import Word2Vec
okt = Okt()

In [ ]:
def Extractor(list) : # keyword는 ['단어', '단어', ...] 형식으로 받는다
    words_n = 3 # 뽑을 키워드 갯수
    keyword = []

    string = okt.nouns(' '.join(list)) # 명사만 찾는다
    string = ' '.join(string) # findall을 하기위해 묶는다
    keyword = ' '.join(re.findall('\w{2,}', string.lower())) # 1글자인 단어들 삭제

    if len(set(keyword)) < words_n :
        words_n = len(set(keyword)) # 단어 갯수가 적을 경우 뽑을 키워드 갯수를 낮춘다

    try :
        key = keywords.keywords(keyword, words = words_n) # 키워드 뽑기
    except :
        words_n -= 1
        key = keywords.keywords(keyword, words = words_n)

    key = key.replace('\n', ' ')
    return key.split(' ')[:words_n]

In [ ]:
train_data = data['cleaned_info'].apply(lambda a : re.findall('[가-힣a-z]+', a)).to_list()

In [ ]:
# train_data = train_data + sent.apply(lambda a : re.findall('[가-힣a-z]+', a)).to_list()

In [ ]:
w2v_model = Word2Vec(sentences = train_data, vector_size = 50, window = 5, min_count = 0, workers = 4, sg = 0)

In [ ]:
def Simmilarity(keyword1, keyword2) : # 문장1의 키워드, 문장2의 키워드를 받는다. keyword2에 사용자 키워드를 가져와야 한다
    vector_dic = {}
    score = 0
    n = len(keyword1)
    for i in [keyword1, keyword2] :
        for ind, keyword in enumerate(i) :
            if keyword in w2v_model.wv.key_to_index.keys() :
                i[ind] = np.round(w2v_model.wv[keyword], 3) # 벡터값을 vector_dic에 저장
            else :
                n -= 1

    for i in keyword1 :
        for j in keyword2 :
            try :
                score += i.dot(j.T) # scaled dot
            except :
                continue
    if n == 0:
        return 0
    return np.round(score / n, 5) # attention score를 다 더하고 query 갯수만큼 나누었다

In [ ]:
Simmilarity(['일기', '기록', '내용'], ['롤플레잉', '캐릭터', '채팅']) # 사용 예시

27.64695

# 실행파일

In [ ]:
with open('pre.txt', 'r') as f:
    dic = []
    for i in f :
        dic.append(i.replace('\n', ''))

dic = pd.DataFrame(dic, columns = ['0'])

In [ ]:
# dic = pd.read_csv('gen_data.csv') # 코딩을 위한 임시 데이터
sent = dic['0']

key_dic = {}
for ind, i in enumerate(sent) :
    i = i.split()
    key_dic[ind] = Extractor(i)

In [ ]:
fav = input('좋아하는 것을 입력해주세요 :')
tel = input('잘하는 것을 입력해주세요 :')

fav = Extractor(fav) # 잘하는 것, 좋아하는 것 키워드 추출
tel = Extractor(tel)

ideal_vec = fav + tel
 # 키워드를 단순하게 합침으로써, 겹치는 키워드가 있다면 유사도 계산 때 더 많은 영향을 줄 것이다
sim_dic = {}
for i in key_dic.keys() :
    if key_dic[i] == ['']:
        sim_dic[i] = 0
        continue
    ideal_vec = fav + tel
    sim_dic[i] = Simmilarity(key_dic[i], ideal_vec)

sorted_dict = sorted(sim_dic.items(), key=lambda item: item[1])

for i, j in sorted_dict[-1:-6:-1] :
    print(dic.iloc[i]['0'])